In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.datasets import VOCDetection
import numpy as np
from torchvision import transforms
from torchvision.datasets import CIFAR10

In [28]:
class CIFARDataset(Dataset):
    def __init__(self, root, train=True, S=7, transform=None, download=True):
        self.dataset = CIFAR10(root=root, train=train, transform=transform, download=download)
        self.S = S
        self.classes = self.dataset.classes
        self.transform = transform
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img, label = self.dataset[idx]

        x_center = 0.5
        y_center = 0.5
        bw = 1.0
        bh = 1.0

        target_tensor = torch.zeros(self.S, self.S, 5 + len(self.classes))
        i = int(y_center * self.S)
        j = int(x_center * self.S)
        target_tensor[i, j, 0:4] = torch.tensor([x_center, y_center, bw, bh])
        target_tensor[i, j, 4] = 1
        target_tensor[i, j, 5 + label] = 1

        if self.transform:
            img = self.transform(img)

        return img, target_tensor   

In [29]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

train_dataset = CIFARDataset(root='./data', train=True, S=7, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

In [30]:
imgs, targets = next(iter(train_loader))
print(imgs.shape)      
print(targets.shape)

TypeError: pic should be PIL Image or ndarray. Got <class 'torch.Tensor'>